# Tutorial 0) Measurement notebook example

The following tutorials all focus on a specific aspect of arbok. This notebook on the other hand gives as few comments as possible and provides structure of a real world measurement notebook.

## 1. Importing modules

### 1.1 Import of arbok modules

In [1]:
from arbok_driver import ArbokDriver, Device, Measurement

from configurations.opx_config import opx_config
from configurations.divider_config import divider_config

2025-09-02 17:56:25,910 - qm - INFO     - Starting session: 8ebfff36-32f4-480f-8fb3-70633b8efafc


In [2]:
mock_device = Device(
    name = 'mock_device',
    opx_config = opx_config,
    divider_config = divider_config)

mock_driver = ArbokDriver(
    name = 'mock_driver',
    device = mock_device
)

In [3]:
mock_measurement = Measurement(
    parent = mock_driver,
    name = "mock_measurement",
    device = mock_device
)

### Populating the measurement

Let's look back on what we did so far! We created a `Device` object that holds the configuration for the OPX as well as a configuration describing the installed dividers on our lines.
That device is then used to create an `ArbokDriver` instance. So far so good but now we want to create some actual measurements.

The fundamental building blocks of arbok are called `SubSequence`s. Those can be arbitrarily simple or complicated.
Below we are inspecting an easy `SquarePulse` class inheriting from `SubSequence`. An element is ramped to a certain amplitude, a wait time is being passed and finally the element is ramped back to its inital voltage. This is assuming the given element is ['sticky'](https://docs.quantum-machines.co/latest/docs/Guides/features/?h=sticky#sticky-element).

You already see that some of the attributes are being called with brackets e.g self.element(), self.ramp_time(), etc. Those will become important in a bit!

In [4]:
import inspect
from IPython.display import Code, display

In [5]:
from example_sequences.square_pulse import SquarePulse
from configurations.square_pulse_config import square_pulse_config

display(Code(inspect.getsource(SquarePulse), language="python"))

class SquarePulse(SubSequence):
    """
    Class containing parameters and sequence for a simple square pulse
    """

    def qua_sequence(self):
        """Macro that will be played within the qua.program() context"""
        qua.align()
        qua.play('ramp'*qua.amp(self.amplitude()), self.element(), duration = self.ramp_time())
        qua.wait(self.t_square_pulse(), self.element())
        qua.play('ramp'*qua.amp(-self.amplitude()), self.element(), duration = self.ramp_time())

In order to populate a measurement with sub-sequences, we have to create a dict reflecting the structure of our measurement. 

In [6]:
sub_sequence_dict = {
    'square_pulse': {
        'sequence': SquarePulse,
        'config': square_pulse_config
    }
}

mock_measurement.add_subsequences_from_dict(sub_sequence_dict)

If you want to check the resulting structure of your measurement you can either list its sub_sequences or you draw the sub_sequence tree.

In [7]:
mock_measurement.sub_sequences

[<SquarePulse: mock_driver_mock_measurement_square_pulse of Measurement: mock_driver_mock_measurement>]

In [8]:
mock_measurement.draw_sub_sequence_tree()

mock_measurement
 └─ square_pulse


Since the arbok_driver is based on QCoDeS, we can use all their helper functions as well to inspect our dynamicaly created instrument. As you see below, so far our driver has one measurement which is implemented as an [`InstrumentModule`](https://microsoft.github.io/Qcodes/api/instrument/index.html#qcodes.instrument.InstrumentModule) which has a sub-module (SquarePulse) itself. `print_readable_snapshot` also lists all available parameters and their cur

In [9]:
mock_driver.print_readable_snapshot()

mock_driver:
	parameter value
--------------------------------------------------------------------------------
IDN       :	None 
iteration :	None 
mock_driver_mock_measurement:
	parameter value
--------------------------------------------------------------------------------
mock_driver_mock_measurement_square_pulse:
	parameter     value
--------------------------------------------------------------------------------
amplitude      :	0.1 (V)
element        :	gate_1 (N/A)
t_square_pulse :	100 (s)


### 1.2 Import self written sequences and configurations

In [10]:
from example_sequences.square_pulse import SquarePulse
from example_sequences.dummy_readout import DummyReadout

NameError: name 'ReadoutPoint' is not defined

In [ ]:
from example_configs.square_pulse_config import square_pulse_conf
from example_configs.dummy_readout_config import dummy_readout_config
from example_configs.dummy_sample import dummy_sample

### 1.3 Generic imports

In [ ]:
import numpy as np
import os
%load_ext rich

## 2. Configuring the driver

### 2.1 Instanciating driver, sequence and set relations

In [ ]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
dummy_sequence = Measurement(qm_driver, 'dummy_sequence', dummy_sample)
qm_driver.add_sequence(dummy_sequence)

In [ ]:
square_pulse = SquarePulse(dummy_sequence, 'square_pulse', dummy_sample, square_pulse_conf)
readout = DummyReadout(dummy_sequence, 'readout', dummy_sample, dummy_readout_config)

### 2.2 Set sweep parameters and values

In [ ]:
dummy_sequence.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    },
    {
        square_pulse.t_square_pulse: np.arange(20, 100, 10, dtype = int)
    }, 
)

### 2.3 Register the gettables to save

Print gettables of a `ReadSequence` to see all available gettables.

In [ ]:
# readout.gettables

In [ ]:
dummy_sequence.register_gettables(
    readout.qubit1.diff(),
    readout.qubit1.read.sensor1_IQ(),
    readout.qubit1.ref.sensor1_IQ(),
)

### 2.4 Compile QUA program and print to file

In [ ]:
qm_driver.dummy_sequence

In [ ]:
qua_program = qm_driver.dummy_sequence.get_qua_program()
qm_driver.print_qua_program_to_file(
    file_name = '0_qua_program.py', 
    qua_program = qua_program,
    add_config=False)

### 2.5 Connect QM and run program

In [ ]:
IP_ADDRESS = '<YOUR_QM_IP>'
qm_driver.connect_opx(IP_ADDRESS)
qm_driver.run(qua_program)

In [ ]:
sweep_list = [
    {qm_driver.iteration: np.arange(100)},
]

## 3. Set up QCoDeS datamanagement and run measurement 

### 3.1 Define qcodes database path

In [ ]:
db_file_path = os.path.join(os.getcwd(), 'example.db')
initialise_or_create_database_at(db_file_path)

### 3.2 Define the experiment and measurement

In [ ]:
MEASUREMENT_NAME = "specific measurement name"

In [ ]:
tutorial_exp = load_or_create_experiment(
    experiment_name="Example Experiment",
    sample_name="dummy_sample"
)
meas = Measurement(exp = tutorial_exp, name = MEASUREMENT_NAME)

### 3.3 Create measurement loop and run the experiment

In [ ]:
@create_measurement_loop(
    sequence = dummy_sequence, measurement=meas, sweep_list=sweep_list)
def run_measurement_loop():
    pass

In [ ]:
run_measurement_loop()